# Vehicle Detection using YOLO with ResNet feature extractor#

** Main video pipeline for the Vehicle Detection project **

#### Imports ####

In [ ]:
import sys
import numpy as np
import cv2
import utils
import matplotlib.pyplot as plt
from model_continue_train import ResNet50
import time
from moviepy.editor import VideoFileClip

%matplotlib inline

### Create & Load Model ###

In [ ]:
%time model = ResNet50(include_top=False, load_weight=True, weights='models/rerun9_0.01_weights.02-2.05.hdf5', input_shape=(224,224,3))

#### Image Processing Pipeline: ####

In [ ]:
def process_img(img):
    dims = (img.shape[1], img.shape[0])
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_float = cv2.resize(img_bgr, (224,224)).astype(np.float32)
    img_float -= 128
    img_in = np.expand_dims(img_float, axis=0)
    
    pred = model.predict(img_in)
    
    bboxes = utils.get_boxes(pred[0], cutoff=0.2, dims=dims)
    bboxes = utils.nonmax_suppression(bboxes, iou_cutoff = 0.05)
    draw = utils.draw_boxes(img, bboxes, color=(0, 255, 0), thick=5, draw_dot=False, radius=3)
    draw = draw.astype(np.uint8)
    
    return draw

In [ ]:
### Test pipeline on image from dataset itself
img = cv2.imread('test_img2.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8,6))
plt.imshow(process_img(img))

In [ ]:
### Test pipeline on image the network has never seen before
img = cv2.imread('test_img.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(8,6))
plt.imshow(process_img(img))

#### Apply to video: ####

In [ ]:
video1 = 'project_video.mp4'
video1_output = 'project_video_output2.mp4'
clip1 = VideoFileClip(video1)#.subclip(30,36)
imgs = clip1.fl_image(process_img)
%time imgs.write_videofile(video1_output, audio=False)